# 🛡️ NLP Toxicity Detection - Live Test

This notebook implements the core logic of the **Low-Latency Moderation API**. It loads the configured toxicity classification model and provides an interactive interface to test messages against the moderation rules.

> **Note:** This notebook acts as a simplified demonstration of the mobile app's moderation system. It allows for viewing and testing the model logic independently and serves as a fallback or debugging tool in case of issues with the main application.

### **How it works**
I have split the implementation into logical blocks to explain each step:
1.  **Install Libraries**: Setting up the environment.
2.  **Imports**: Loading necessary modules.
3.  **Configuration**: defining model parameters.
4.  **Services**: Loading the AI model.
5.  **Logic**: Business rules for blocking/allowing messages.
6.  **Interactive UI**: A simple testing widget.

### 1. Install Necessary Libraries
I start by installing the external libraries required for this project/demo.
*   `transformers`: Provides the pre-trained NLP models (Hugging Face) for text classification.
*   `pydantic`: Used for defining data models ensuring my inputs and outputs are strictly typed and valid.
*   `ipywidgets`: Enables interactive UI controls (like text boxes and buttons) directly within this notebook.

In [ ]:
# 1. Install necessary libraries
!pip install -q transformers pydantic ipywidgets

### 2. Imports
Here I import the standard and third-party modules.
*   `os`, `functools`, `typing`: Standard Python utilities for file paths, caching, and type hinting.
*   `transformers`: Classes to load the specific model architecture (`AutoModelForSequenceClassification`) and tokenizer (`AutoTokenizer`).
*   `pydantic`: `BaseModel` to create my structured data classes.
*   `ipywidgets` & `IPython.display`: For building the user interface.

In [ ]:
# 2. Imports
import os
from typing import Literal, TypedDict
from functools import lru_cache
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline
from pydantic import BaseModel, Field
import ipywidgets as widgets
from IPython.display import display, clear_output

### 3. Configuration & Data Models
I define a `Settings` class to centralize configuration (like the specific model name and confidence threshold).

I also define Pydantic models:
*   `ToxicityResult`: Captures the raw output from the model (label and confidence).
*   `ModerationResponse`: Represents the final decision sent back to the application (status accepted/rejected, message, etc.). Structure ensures consistency across the app.

In [ ]:
# --- Configuration ---
class Settings:
    model_name: str = "martin-ha/toxic-comment-model"
    negative_threshold: float = 0.70

settings = Settings()

# --- Models ---
class ToxicityResult(BaseModel):
    label: Literal["TOXIC", "NON-TOXIC"]
    confidence: float

class ModerationResponse(BaseModel):
    status: Literal["accepted", "rejected"]
    message: str
    label: Literal["TOXIC", "NON-TOXIC"]
    confidence: float

### 4. Model Loading & Services
I load the pre-trained model and tokenizer from Hugging Face.
*   I use `AutoTokenizer` and `AutoModelForSequenceClassification` which automatically detect the correct architecture for "martin-ha/toxic-comment-model".
*   I wrap the prediction logic in `classify_message` to handle the raw pipeline call and format it into the `ToxicityResult` model.

In [ ]:
# --- Services ---
print(f"🔄 Loading model: {settings.model_name}...")

# Load Model & Tokenizer (Cached globally)
try:
    tokenizer = AutoTokenizer.from_pretrained(settings.model_name)
    model = AutoModelForSequenceClassification.from_pretrained(settings.model_name)
    pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer)
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f" ❌ Error loading model: {e}")
    pipeline = None

def classify_message(text: str) -> ToxicityResult:
    """Runs toxicity classification on text."""
    if not pipeline:
        raise RuntimeError("Model pipeline not initialized.")
    
    # Run prediction
    result = pipeline(text)[0]
    raw_label = result["label"].upper()
    confidence = float(result["score"])
    
    return ToxicityResult(label=raw_label, confidence=confidence)

### 5. Main Moderation Logic
The `check_message` function contains the business rules.
*   It calls `classify_message` to get the raw probability.
*   **Blocking Rule**: If the text is toxic AND confidence is high (≥ 0.70), I reject it.
*   **Flagging Rule**: If toxic but lower confidence, I might log it but ultimately still pass it (or reject depending on policy). Here, I default to setting status to "rejected" for toxic labels but distinguishing slightly in the message or final handling.

In [ ]:
# --- Main Logic ---
def check_message(text: str) -> ModerationResponse:
    """Classify the incoming text and block highly confident toxic content."""
    result = classify_message(text)

    # BLOCK if TOXIC and High Confidence (>= 0.70)
    if result.label == "TOXIC" and result.confidence >= settings.negative_threshold:
        return ModerationResponse(
            status="rejected",
            message="⛔ Message classified as toxic.",
            label=result.label,
            confidence=result.confidence,
        )

    # Note: Even if TOXIC (but low confidence), rejected but message to passed.
    status = "accepted"
    return ModerationResponse(
        status=status,
        message="✅ Message passed moderation.",
        label=result.label,
        confidence=result.confidence,
    )


### 6. Interactive Interface
Finally, I use `ipywidgets` to create a simple text area and button. This allows you to test the model immediately without needing a frontend application.
*   `on_button_click` handles the interaction, runs the check, and prints a user-friendly result.

In [ ]:
# --- Interactive UI ---
input_box = widgets.Textarea(
    value='',
    placeholder='Type a message here to test moderation...',
    description='Message:',
    layout=widgets.Layout(width='60%', height='100px')
)

check_button = widgets.Button(
    description='Check Toxicity',
    button_style='primary', # 'success', 'info', 'warning', 'danger'
    icon='check'
)

output_area = widgets.Output()

def on_button_click(b):
    with output_area:
        clear_output()
        message = input_box.value.strip()
        if not message:
            print("⚠️ Please enter a message first.")
            return
            
        try:
            print(f"Analyzing: '{message}' ...\n")
            response = check_message(message)
            
            # Display Result
            print(f"--- RESULT ---")
            print(f"Status:     {response.status.upper()}")
            print(f"Label:      {response.label}")
            print(f"Confidence: {response.confidence:.4f}")
            print(f"Response:   {response.message}")
            
            # Visual Indicator
            if response.status == "rejected" and response.confidence >= settings.negative_threshold:
                 print("\n🔴 BLOCKED (High Confidence)")
            elif response.label == "TOXIC":
                 print("\n🟠 FLAGGED (Low Confidence)")
            else:
                 print("\n🟢 CLEAN")
                 
        except Exception as e:
            print(f"Error: {e}")

check_button.on_click(on_button_click)

# Display the widget
display(widgets.VBox([input_box, check_button, output_area]))